# 1. Data Loading and Initial Inspection

In [1]:
import pandas as pd

df = pd.read_csv("/content/healthcare-dataset-stroke-data.csv")

print(df.info())
print(df.isna().sum())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB
None
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
wo

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


# 2. Data Cleaning and Preprocessing

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


df = df.drop(columns=["id"])


df["bmi"] = df["bmi"].fillna(df["bmi"].median())


X = df.drop("stroke", axis=1)
y = df["stroke"]


X = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


num_cols = ["age", "avg_glucose_level", "bmi"]
scaler = StandardScaler()

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

print(X_train.shape, X_test.shape)


(3577, 16) (1533, 16)


# 3. Baseline Modeling Without Class Balancing

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

lr = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)

print("Logistic Regression ROC-AUC:",
      roc_auc_score(y_test, lr.predict_proba(X_test)[:,1]))

print("Random Forest ROC-AUC:",
      roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))


Logistic Regression ROC-AUC: 0.8378417924096937
Random Forest ROC-AUC: 0.8107178783721993


#4. Class-Weighted Learning

In [4]:
lr_w = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

lr_w.fit(X_train, y_train)

print("Class-weighted LR ROC-AUC:",
      roc_auc_score(y_test, lr_w.predict_proba(X_test)[:,1]))

print(classification_report(y_test, lr_w.predict(X_test)))


Class-weighted LR ROC-AUC: 0.8388660265203475
              precision    recall  f1-score   support

           0       0.99      0.73      0.84      1458
           1       0.13      0.79      0.22        75

    accuracy                           0.73      1533
   macro avg       0.56      0.76      0.53      1533
weighted avg       0.94      0.73      0.81      1533



# 5. Data Balancing with SMOTE

In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

pd.Series(y_train_bal).value_counts()

,count
stroke,
0,3403
1,3403


#6. Model Training on Balanced Data

In [6]:
lr_bal = LogisticRegression(max_iter=1000, random_state=42)
rf_bal = RandomForestClassifier(n_estimators=100, random_state=42)

lr_bal.fit(X_train_bal, y_train_bal)
rf_bal.fit(X_train_bal, y_train_bal)

print("LR (SMOTE) ROC-AUC:",
      roc_auc_score(y_test, lr_bal.predict_proba(X_test)[:,1]))

print("RF (SMOTE) ROC-AUC:",
      roc_auc_score(y_test, rf_bal.predict_proba(X_test)[:,1]))


LR (SMOTE) ROC-AUC: 0.8162962962962963
RF (SMOTE) ROC-AUC: 0.7909830818472794


# 7. Global Model Interpretation with SHAP

In [14]:
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt


X_train_clean = X_train.astype(float)
X_test_clean  = X_test.astype(float)


X_train_np = X_train_clean.to_numpy(dtype=np.float64)
X_test_np  = X_test_clean.to_numpy(dtype=np.float64)

feature_names = X_train_clean.columns.tolist()

assert not np.isnan(X_train_np).any(), "NaN found in X_train"
assert not np.isinf(X_train_np).any(), "Inf found in X_train"


masker = shap.maskers.Independent(X_train_np)


explainer = shap.Explainer(
    lr_bal.predict_proba,
    masker,
    feature_names=feature_names
)


shap_values = explainer(X_test_np[:5000])
shap_class1 = shap_values[..., 1]


shap.plots.bar(
    shap_class1,
    max_display=10,
    show=False
)
plt.savefig("figures/shap_global_bar.png", bbox_inches="tight")
plt.close()


shap.plots.beeswarm(
    shap_class1,
    max_display=10,
    show=False
)
plt.savefig("figures/shap_global_beeswarm.png", bbox_inches="tight")
plt.close()



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature na

#8. Local SHAP Explanation (Highest-Risk Patient)

In [16]:
import numpy as np
import shap
import matplotlib.pyplot as plt

y_pred_proba = lr_bal.predict_proba(X_test_clean)[:, 1]
idx = np.argmax(y_pred_proba)

patient_np = X_test_np[idx:idx+1]
patient_df = X_test_clean.iloc[idx:idx+1]

shap_patient = explainer(patient_np)


shap_patient_class1 = shap_patient[..., 1]


shap.plots.bar(
    shap_patient_class1,
    max_display=10,
    show=False
)
plt.savefig(
    "figures/shap_local_highest_risk.png",
    bbox_inches="tight"
)
plt.close()


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist